# Generate data for `test_correct_count_rate`

In [ ]:
import os
import pprint

from astropy.io import fits
import numpy as np

from jwst_magic.fsw_file_writer import buildfgssteps

In [ ]:
__location__ = '/Users/lchambers/TEL/Commissioning/Tools/fgs-commissioning/jwst_magic/tests/'
ROOT = "test_buildfgssteps"
SEGMENT_INFILE = os.path.join(__location__, 'data', '{}_ALLpsfs.txt'.format(ROOT))
SELECTED_SEGS = os.path.join(__location__, 'data', '{}_regfile.txt'.format(ROOT))

In [ ]:
with fits.open(
    '/Users/lchambers/TEL/Commissioning/Tools/fgs-commissioning/jwst_magic/tests/data/fgs_data_2_cmimf.fits'
) as hdulist:
    open_image = hdulist[1].data

In [ ]:
1 ID
bias : [4113.857078887622, 104049.3416361762]
norm : [-1.7679394239994082e-20, 5283.94677734375]
strips : [4113.857078887622, 65535.0]
cds : [-16.6, 65000.0]
product : [4113.857078887622, 65535.0]
countrates: [65596. 91475. 91549. 95731. 95747.]

In [ ]:
d = {}
for guider in [1, 2]:
    d[guider] = {}
    for step in ['LOSTRK', 'CAL', 'ID', 'ACQ1', 'ACQ2', 'TRK']:
        d[guider][step] = {}
        
        bias = []
        norm = []
        strips = []
        cds = []
        product = []
        
        if step != 'LOSTRK':
            n = 10
        else:
            n = 1
            
        for i in range(n):
            # Run the code
            BFS = buildfgssteps.BuildFGSSteps(open_image, guider, 'test', step, guiding_selections_file=SELECTED_SEGS,
                                out_dir=__location__, catalog=SEGMENT_INFILE);

            # Don't know how to test bias - since it is so variable
            if step is not 'LOSTRK':
                bias.append([np.min(BFS.bias), np.max(BFS.bias)])

            norm.append([np.min(BFS.time_normed_im), np.max(BFS.time_normed_im)])
            if step is 'ID':
                strips.append([np.min(BFS.strips), np.max(BFS.strips)])
            if 'TRK' not in step:
                cds.append([np.min(BFS.cds), np.max(BFS.cds)])
            product.append([np.min(BFS.image), np.max(BFS.image)])
        
        print()
        print(guider, step)
        print()
        for l, name in zip([bias, norm , strips , cds , product, BFS.countrate], 
                              ['bias', 'norm' , 'strips' , 'cds' , 'product', 'countrate']):
            if (l != [] and step == 'LOSTRK') or (name in ['countrate', 'norm']):
                d[guider][step][name] = l[0]
            elif l != [] and name != 'countrate':
#                 print(name, ': [{}, {}]'.format(np.average([m[0] for m in l]), np.average([m[1] for m in l])))
#                 print(name)
#                 print(l)
#                 print()
#                 print(np.average([m[0] for m in l]))
#                 print(np.average([m[1] for m in l]))
                d[guider][step][name] = [round(np.average([m[0] for m in l]), -1), 
                                         round(np.average([m[1] for m in l]), -1)]
#             elif name == 'countrate':
#                 d[guider][step][name] = l
#         print('countrates:', BFS.countrate)
        

In [ ]:
pprint.pprint(d)

# Generate data for `coordinate_tranforms`

In [ ]:
import inspect
from jwst_magic.utils import coordinate_transforms

In [ ]:
for func in sorted(dir(coordinate_transforms)):
    if not func.startswith('_') and '2' in func:
        print(func)
        func_to_call = getattr(coordinate_transforms, func)
#         result = func_to_call()
        arg_names = inspect.getfullargspec(func_to_call)[0]
#         print(arg_names)
    
        if 'x_raw' in arg_names:
            x, y = (1743.3, 241.9)
            
        else:
            x, y = (-55.736935, 8.139518)
            
        if 'guider' in arg_names:
            for guider in range(2):
                result = func_to_call(x, y, guider + 1)
                print(result)
        else:
            result = func_to_call(x, y)
            print(result)

In [ ]:
method_to_call = getattr(foo, 'bar')
result = method_to_call()

In [ ]:
ANGLE_COORDS = (-55.736935, 8.139518)
coordinate_transforms.DHAS2Raw(*ANGLE_COORDS, 1)